# Конвертація Keras → TensorFlow.js

Цей ноутбук:
- перевіряє, що є `artifacts/keras_model.keras` і `artifacts/scaler.json`
- запускає `tensorflowjs_converter`
- отримує `web/model/model.json` + `*.bin`
- копіює `scaler.json` у `web/scaler.json` (щоб працювало на GitHub Pages)


In [ ]:
# !pip install tensorflowjs==4.22.0

from pathlib import Path
import shutil
import subprocess
import json

ART = Path("..") / "artifacts"
IN_MODEL = ART / "keras_model.keras"
IN_SCALER = ART / "scaler.json"

WEB = Path("..") / "web"
OUT_MODEL = WEB / "model"
OUT_SCALER = WEB / "scaler.json"

IN_MODEL.exists(), IN_SCALER.exists()


## 1) Перевірка scaler.json

In [ ]:
scaler = json.loads(IN_SCALER.read_text(encoding="utf-8"))
required = ["mean","scale","features","input_hours","horizon"]
missing = [k for k in required if k not in scaler]
missing


## 2) Конвертація (tensorflowjs_converter)

In [ ]:
OUT_MODEL.mkdir(parents=True, exist_ok=True)

cmd = [
    "tensorflowjs_converter",
    "--input_format=keras",
    "--output_format=tfjs_layers_model",
    str(IN_MODEL),
    str(OUT_MODEL),
]
print("Running:", " ".join(cmd))
subprocess.check_call(cmd)


## 3) Копія scaler.json у web/

In [ ]:
shutil.copy2(IN_SCALER, OUT_SCALER)
print("Copied scaler to:", OUT_SCALER)


## 4) Перевірка вихідних файлів

In [ ]:
model_json = OUT_MODEL / "model.json"
shards = list(OUT_MODEL.glob("group*-shard*.bin"))
model_json.exists(), len(shards), shards[:3]


## Готово ✅
Тепер можна запускати статичний сервер і відкривати `web/index.html`.

In [ ]:
# Локальний запуск (з кореня проекту):
# python -m http.server 8000
# Відкрити: http://localhost:8000/web/
